# TSP 求解器消融实验

本 Notebook 用于评估 TSP 求解器中不同算法组件的贡献。

## 实验配置

我们将测试以下配置：
1. **完整版（Baseline）**：所有组件启用
2. **无 EAX**：禁用 EAX Lite 交叉，强制使用 SCX
3. **无 Scout**：禁用多进程 Scout worker
4. **无 RTR**：禁用 RTR 替换策略，使用简单最差替换
5. **无 GLS**：禁用 Guided Local Search 惩罚机制

In [1]:
import os
import sys
import time
import json
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 支持中文显示
matplotlib.rcParams['axes.unicode_minus'] = False  # 正确显示负号

# 导入修改后的 test_performance 模块
import test_performance

print("环境设置完成")

环境设置完成


## 定义实验配置

In [2]:
# 测试数据集
TEST_FILE = "tour750.csv"  # 中等规模，运行时间适中
MAX_TIME = 180  # 每个配置最多运行 90 秒

# 定义所有消融实验配置
ABLATION_CONFIGS = {
    'baseline': {
        'name': '完整版 (Baseline)',
        'config': {
            'use_eax': True,
            'use_scout': True,
            'use_rtr': True,
            'use_gls': True,
        },
        'description': '所有组件启用'
    },
    'no_eax': {
        'name': '无 EAX',
        'config': {
            'use_eax': False,
            'use_scout': True,
            'use_rtr': True,
            'use_gls': True,
        },
        'description': '禁用 EAX Lite，强制使用 SCX 交叉'
    },
    'no_scout': {
        'name': '无 Scout',
        'config': {
            'use_eax': True,
            'use_scout': False,
            'use_rtr': True,
            'use_gls': True,
        },
        'description': '禁用 Scout 多进程搜索'
    },
    'no_rtr': {
        'name': '无 RTR',
        'config': {
            'use_eax': True,
            'use_scout': True,
            'use_rtr': False,
            'use_gls': True,
        },
        'description': '禁用 RTR 替换策略，使用简单最差替换'
    },
    'no_gls': {
        'name': '无 GLS',
        'config': {
            'use_eax': True,
            'use_scout': True,
            'use_rtr': True,
            'use_gls': False,
        },
        'description': '禁用 GLS 惩罚机制'
    },
}

print(f"实验配置数量: {len(ABLATION_CONFIGS)}")
print(f"测试数据集: {TEST_FILE}")
print(f"单个配置最大运行时间: {MAX_TIME} 秒")

实验配置数量: 5
测试数据集: tour750.csv
单个配置最大运行时间: 180 秒


## 运行消融实验

In [ ]:
# 创建日志目录
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
log_dir = f"logs/ablation_study_{timestamp}"
os.makedirs(log_dir, exist_ok=True)

print(f"日志目录: {log_dir}")

# 存储所有实验结果
results = {}

# 运行每个配置
for config_id, config_info in ABLATION_CONFIGS.items():
    print(f"\n{'='*60}")
    print(f"运行配置: {config_info['name']}")
    print(f"描述: {config_info['description']}")
    print(f"配置参数: {config_info['config']}")
    print(f"{'='*60}")
    
    # 初始化求解器
    solver = test_performance.r0927480(config=config_info['config'])
    
    # 记录开始时间
    start_time = time.time()
    
    try:
        # 运行优化（注意：Reporter 会控制实际运行时间）
        result = solver.optimize(TEST_FILE)
        
        # 记录结束时间
        elapsed_time = time.time() - start_time
        
        # 提取结果
        if result is not None:
            results[config_id] = {
                'name': config_info['name'],
                'config': config_info['config'],
                'description': config_info['description'],
                'best_fitness': float(result['best_fitness']),
                'generations': int(result['generations']),
                'elapsed_time': elapsed_time,
                'metrics': result.get('metrics', {})
            }
            
            print(f"\n✓ 完成 - 最优解: {result['best_fitness']:.2f}, 代数: {result['generations']}, 用时: {elapsed_time:.1f}s")
        else:
            print(f"\n✗ 优化返回 None（Reporter 控制）")
            results[config_id] = {
                'name': config_info['name'],
                'config': config_info['config'],
                'description': config_info['description'],
                'best_fitness': None,
                'generations': None,
                'elapsed_time': elapsed_time,
                'metrics': {}
            }
    
    except Exception as e:
        print(f"\n✗ 错误: {str(e)}")
        import traceback
        traceback.print_exc()
        results[config_id] = {
            'name': config_info['name'],
            'config': config_info['config'],
            'description': config_info['description'],
            'error': str(e),
            'elapsed_time': time.time() - start_time
        }

# 保存结果到 JSON
results_file = os.path.join(log_dir, 'ablation_results.json')
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"\n\n所有实验完成！结果已保存到: {results_file}")

日志目录: logs/ablation_study_20251228_023605

运行配置: 完整版 (Baseline)
描述: 所有组件启用
配置参数: {'use_eax': True, 'use_scout': True, 'use_rtr': True, 'use_gls': True}


## 结果可视化

In [ ]:
# 准备数据
config_names = []
best_fitnesses = []
generations_list = []
elapsed_times = []

for config_id in ['baseline', 'no_eax', 'no_scout', 'no_rtr', 'no_gls']:
    if config_id in results and results[config_id].get('best_fitness') is not None:
        config_names.append(results[config_id]['name'])
        best_fitnesses.append(results[config_id]['best_fitness'])
        generations_list.append(results[config_id]['generations'])
        elapsed_times.append(results[config_id]['elapsed_time'])

# 创建子图
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'消融实验结果对比 ({TEST_FILE})', fontsize=16, fontweight='bold')

# 1. 最优解质量对比
ax1 = axes[0, 0]
bars1 = ax1.bar(range(len(config_names)), best_fitnesses, color='steelblue', alpha=0.8)
ax1.set_xticks(range(len(config_names)))
ax1.set_xticklabels(config_names, rotation=15, ha='right')
ax1.set_ylabel('最优解长度', fontsize=12)
ax1.set_title('(a) 最优解质量', fontsize=12, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# 在柱子上显示数值
for i, (bar, val) in enumerate(zip(bars1, best_fitnesses)):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(best_fitnesses)*0.01, 
             f'{val:.1f}', ha='center', va='bottom', fontsize=9)

# 2. 性能下降百分比（相对于 baseline）
if len(best_fitnesses) > 0:
    baseline_fitness = best_fitnesses[0]  # 假设第一个是 baseline
    degradations = [(f - baseline_fitness) / baseline_fitness * 100 for f in best_fitnesses[1:]]
    
    ax2 = axes[0, 1]
    colors = ['red' if d > 0 else 'green' for d in degradations]
    bars2 = ax2.bar(range(len(degradations)), degradations, color=colors, alpha=0.7)
    ax2.set_xticks(range(len(degradations)))
    ax2.set_xticklabels(config_names[1:], rotation=15, ha='right')
    ax2.set_ylabel('性能变化 (%)', fontsize=12)
    ax2.set_title('(b) 相对于完整版的性能变化', fontsize=12, fontweight='bold')
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax2.grid(axis='y', alpha=0.3)
    
    # 显示百分比数值
    for i, (bar, val) in enumerate(zip(bars2, degradations)):
        ax2.text(bar.get_x() + bar.get_width()/2, 
                 bar.get_height() + (0.5 if val > 0 else -0.5), 
                 f'{val:+.2f}%', ha='center', 
                 va='bottom' if val > 0 else 'top', fontsize=9)

# 3. 迭代代数对比
ax3 = axes[1, 0]
bars3 = ax3.bar(range(len(config_names)), generations_list, color='coral', alpha=0.8)
ax3.set_xticks(range(len(config_names)))
ax3.set_xticklabels(config_names, rotation=15, ha='right')
ax3.set_ylabel('迭代代数', fontsize=12)
ax3.set_title('(c) 迭代代数对比', fontsize=12, fontweight='bold')
ax3.grid(axis='y', alpha=0.3)

for i, (bar, val) in enumerate(zip(bars3, generations_list)):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(generations_list)*0.01, 
             f'{val}', ha='center', va='bottom', fontsize=9)

# 4. 运行时间对比
ax4 = axes[1, 1]
bars4 = ax4.bar(range(len(config_names)), elapsed_times, color='darkseagreen', alpha=0.8)
ax4.set_xticks(range(len(config_names)))
ax4.set_xticklabels(config_names, rotation=15, ha='right')
ax4.set_ylabel('运行时间 (秒)', fontsize=12)
ax4.set_title('(d) 运行时间对比', fontsize=12, fontweight='bold')
ax4.grid(axis='y', alpha=0.3)

for i, (bar, val) in enumerate(zip(bars4, elapsed_times)):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(elapsed_times)*0.01, 
             f'{val:.1f}s', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plot_file = os.path.join(log_dir, 'ablation_comparison.png')
plt.savefig(plot_file, dpi=300, bbox_inches='tight')
print(f"\n对比图已保存到: {plot_file}")
plt.show()

## 组件贡献分析

In [ ]:
# 打印详细的组件贡献分析
print("\n" + "="*70)
print("组件贡献分析报告")
print("="*70)

if 'baseline' in results and results['baseline'].get('best_fitness') is not None:
    baseline_fitness = results['baseline']['best_fitness']
    
    print(f"\n基准配置 (完整版):")
    print(f"  最优解: {baseline_fitness:.2f}")
    print(f"  代数: {results['baseline']['generations']}")
    print(f"  用时: {results['baseline']['elapsed_time']:.1f}s")
    
    print("\n" + "-"*70)
    print("各组件消融后的影响:")
    print("-"*70)
    
    # 按性能下降排序
    ablation_impacts = []
    for config_id in ['no_eax', 'no_scout', 'no_rtr', 'no_gls']:
        if config_id in results and results[config_id].get('best_fitness') is not None:
            fitness = results[config_id]['best_fitness']
            degradation = (fitness - baseline_fitness) / baseline_fitness * 100
            ablation_impacts.append({
                'name': results[config_id]['name'],
                'fitness': fitness,
                'degradation': degradation,
                'description': results[config_id]['description']
            })
    
    # 按性能下降排序（从大到小）
    ablation_impacts.sort(key=lambda x: x['degradation'], reverse=True)
    
    for i, impact in enumerate(ablation_impacts, 1):
        print(f"\n{i}. {impact['name']}")
        print(f"   {impact['description']}")
        print(f"   最优解: {impact['fitness']:.2f}")
        print(f"   性能变化: {impact['degradation']:+.2f}%")
        
        # 根据性能下降判断组件重要性
        if impact['degradation'] > 5:
            importance = "🔴 非常重要"
        elif impact['degradation'] > 2:
            importance = "🟡 重要"
        elif impact['degradation'] > 0:
            importance = "🟢 有贡献"
        else:
            importance = "⚪ 无明显影响或提升"
        print(f"   重要性评估: {importance}")
    
    print("\n" + "="*70)
    print("结论:")
    print("="*70)
    print(f"最重要的组件: {ablation_impacts[0]['name']} (性能下降 {ablation_impacts[0]['degradation']:.2f}%)")
    print(f"影响最小的组件: {ablation_impacts[-1]['name']} (性能变化 {ablation_impacts[-1]['degradation']:+.2f}%)")
else:
    print("\n⚠️ 基准配置未成功运行，无法生成分析报告")

print("\n消融实验完成！")

## 指标详细对比

In [ ]:
# 提取和对比关键指标
print("\n" + "="*80)
print("关键指标详细对比")
print("="*80)

# 定义要对比的指标
metrics_to_compare = [
    ('eax_attempts', 'EAX 尝试次数'),
    ('eax_success', 'EAX 成功次数'),
    ('scx_fallback', 'SCX 回退次数'),
    ('scout_sent', 'Scout 发送次数'),
    ('scout_received', 'Scout 接收次数'),
    ('scout_improved', 'Scout 改进次数'),
    ('rtr_accepts', 'RTR 接受次数'),
    ('rtr_rejects', 'RTR 拒绝次数'),
    ('gls_activations', 'GLS 激活次数'),
    ('restart_count', '重启次数'),
]

# 打印表头
print(f"\n{'指标':<20}", end='')
for config_id in ['baseline', 'no_eax', 'no_scout', 'no_rtr', 'no_gls']:
    if config_id in results:
        short_name = results[config_id]['name'].replace('完整版 (Baseline)', 'Baseline')
        print(f"{short_name:>12}", end='')
print()
print("-" * 80)

# 打印每个指标
for metric_key, metric_name in metrics_to_compare:
    print(f"{metric_name:<20}", end='')
    for config_id in ['baseline', 'no_eax', 'no_scout', 'no_rtr', 'no_gls']:
        if config_id in results and 'metrics' in results[config_id]:
            value = results[config_id]['metrics'].get(metric_key, 0)
            if isinstance(value, (int, float)):
                print(f"{value:>12}", end='')
            else:
                print(f"{'N/A':>12}", end='')
        else:
            print(f"{'N/A':>12}", end='')
    print()

print("="*80)